In [1]:
# imports

from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
import pandas as pd
from collections import defaultdict
import re

In [2]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
load_dotenv(dotenv_path=dotenv_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# main constants

GPT_MODEL_TEXT_ALIAS = "gpt-4-turbo-preview" # points to latest GPT model
GPT_MODEL_TEXT = "gpt-4-0125-preview"


In [114]:
# variations
n = 1 # number of trisociations in a run
runs = 10 # number of runs
temperature = 1.3 # temperature setting for OpenAI API

# a temperature of 1.5 is unreliable at n > 50 and run >=1
# a temperature of 1.3 is unreliable at n > 300 and run >=1

In [115]:
# base prompt
base_prompt = f"""
Generate {n} trisociations in the format below. No additional comments or elaborations. Avoid duplicate words across trisociations.
1. Word - Word - Word
2. Word - Word - Word
...
"""

In [44]:
#instantiate client
client = OpenAI()

In [116]:
# Function to perform a single run and collect data
def perform_run(prompt_text):
    start_time = time.time()
    completion = client.chat.completions.create(
        model=GPT_MODEL_TEXT_ALIAS,
        temperature=temperature,
        messages=[
            {"role": "system", "content": prompt_text},
        ]
    )
    end_time = time.time()
    
    trisociation_answer = completion.choices[0].message.content
    execution_time = end_time - start_time
    prompt_tokens = completion.usage.prompt_tokens
    completion_tokens = completion.usage.completion_tokens
    total_tokens = completion.usage.total_tokens

    return {
        'execution_time': execution_time,
        'prompt_tokens': prompt_tokens,
        'completion_tokens': completion_tokens,
        'total_tokens': total_tokens,
        'trisociations': trisociation_answer
    }

In [120]:
# List to store results from all runs
results = []

# Perform multiple runs
for _ in range(runs):
    result = perform_run(base_prompt)
    results.append(result)

# Additionally, print each run's trisociations
for i, r in enumerate(results, 1):
    print(f"Run {i} trisociations:\n{r['trisociations']}\n")

Run 1 trisociations:
1. Thimble - Galaxy - Twirl

Run 2 trisociations:
Hedgehog - Flamboyant - Spaceship

Run 3 trisociations:
Gooseberry - Marathon - Incense

Run 4 trisociations:
1. Library - Galaxy - Sandal

Run 5 trisociations:
1. Eclipse - Maritime - Encryption

Run 6 trisociations:
1. Maple - Algorithm - Saddle

Run 7 trisociations:
Beach - Encryption - Volcano

Run 8 trisociations:
Dinosaur - Saturn - Piano

Run 9 trisociations:
Alaska - Piano - Quantum Mechanics

Run 10 trisociations:
1. Gravity - Raven - Guitar



In [112]:
# function to calculate and display per run time and token metrics
def calculate_and_display_metrics(results):
    # Prepare data for DataFrame
    data = [{
        'Execution Time': result['execution_time'],
        'Prompt Tokens': result['prompt_tokens'],
        'Completion Tokens': result['completion_tokens'],
        'Total Tokens': result['total_tokens']
    } for result in results]
    
    # Create DataFrame
    df_metrics = pd.DataFrame(data)
    
    # Set custom index names (Run 1, Run 2, ...)
    run_indices = [f'Run {i+1}' for i in range(len(results))]
    df_metrics.index = run_indices

    # Apply styling for better readability
    styled_df = df_metrics.style.format({
        'Execution Time': "{:.2f} seconds",
        'Prompt Tokens': "{:.0f}",
        'Completion Tokens': "{:.0f}",
        'Total Tokens': "{:.0f}"
    }).background_gradient(cmap='viridis', subset=['Total Tokens'])

    return styled_df



metrics_df = calculate_and_display_metrics(results)
metrics_df

,Execution Time,Prompt Tokens,Completion Tokens,Total Tokens
Run 1,17.31 seconds,53,463,516


In [113]:
# function to get duplicate stats per run
def per_run_aggregate_summary(results):
    detailed_data = []
    for run_index, result in enumerate(results, start=1):
        trisociations = result['trisociations'].strip()
        trisociation_list = trisociations.split('\n')

        for trisociation_index, trisociation in enumerate(trisociation_list, start=1):
            words = re.sub(r'^\d+\.\s*', '', trisociation).replace(' - ', ' ').split()
            for word in words:
                detailed_data.append({'run': run_index, 'word': word})

    df = pd.DataFrame(detailed_data)

    summary_stats = []
    unique_runs = df['run'].unique()
    
    for run in sorted(unique_runs):
        run_df = df[df['run'] == run]
        
        total_words = run_df.shape[0]
        total_unique_words = run_df['word'].nunique()
        total_duplicates = run_df[run_df.duplicated('word', keep=False)]['word'].nunique()
        
        if not run_df[run_df.duplicated('word', keep=False)].empty:
            mode_duplicate = run_df[run_df.duplicated('word', keep=False)]['word'].value_counts().idxmax()
            max_dup_freq = run_df[run_df.duplicated('word', keep=False)]['word'].value_counts().max()
            modal_dup_freq = run_df[run_df.duplicated('word', keep=False)]['word'].value_counts().mode()[0]
        else:
            mode_duplicate = 'None'
            max_dup_freq = 0
            modal_dup_freq = 0
        
        min_dup_freq = run_df[run_df.duplicated('word', keep=False)]['word'].value_counts().min() if not run_df[run_df.duplicated('word', keep=False)].empty else 0
        
        summary_stats.append({
            'Total Words': total_words,
            'Total Unique Words': total_unique_words,
            'Total Duplicates': total_duplicates,
            'Max Dup Word': mode_duplicate,
            'Max Dup Freq': max_dup_freq,
            'Modal Dup Freq': modal_dup_freq,
            'Min Dup Freq': min_dup_freq
        })
    
    summary_df = pd.DataFrame(summary_stats, index=[f'Run {run}' for run in sorted(unique_runs)])
    
    # Apply styling
    styled_df = summary_df.style\
        .format({'Total Words': "{:}", 'Total Unique Words': "{:}", 'Total Duplicates': "{:}", 
                 'Max Dup Word': "{}", 'Max Dup Freq': "{:}", 'Modal Dup Freq': "{:}", 'Min Dup Freq': "{:}"})\
        .set_table_styles([{
            'selector': 'th',
            'props': [
                ('background-color', '#f4f4f4'),
                ('color', '#6d6d6d'),
                ('font-weight', 'bold')
            ]}])\
        .set_caption("Summary Statistics per Run")\
        .set_properties(**{'text-align': 'left'})

    return styled_df

# display styled DataFrame
per_run_aggregate_summary_styled = per_run_aggregate_summary(results)
per_run_aggregate_summary_styled


,Total Words,Total Unique Words,Total Duplicates,Max Dup Word,Max Dup Freq,Modal Dup Freq,Min Dup Freq
Run 1,158,151,7,Forest,2,2,2
